# Simrunner - Example Use

Author: Tom Norman  
Version 0.1.0 (Prerelease)

__Disclaimer__: 
This package, or any of the work associated with simrunner has absolutely no affiliation with Tuflow or BMT. 

The example use utilises the demostration version of Tuflow, available [here](https://downloads.tuflow.com/TUFLOW/Wiki_Example_Models/TUFLOW_Example_Model_Dataset.zip). The way the demostration folder and TCFs are setup is not ideal for a production environment as it has all the examples lumped together, so to follow along, pull out 'EG16' with run number '006' and put this into its own project directory. 

Simrunner requires that the TCFs are in the following format.  

<span style="color:red">GROUP ID</span>\_<span style="color:orange">ARGUMENTS</span>\_<span style="color:green">RUN NUMBER</span>  

All TCFs with the same group id need the same set of run arguments, otherwise errors will occur. In the case of EG16 demostration:
- group id = 'EG16', 
- arguments = '-s1, -s2, -e1, -e2'
- run number = '006'

Import Simrunner to use.

In [1]:
import simrunner.tuflow as tfr
import time

Set the parameters

In [2]:
parameters: tfr.Parameters = tfr.Parameters(
    exec_path = r"..\tests\data\tuflow\examples\production\exe",  # The path to the TUFLOW executable.
    root = r"..\tests\data\tuflow\examples\production\runs",      # The root of the model, location of the .tcf file.
    version = r"2023-03-AC",                                      # The version of the engine.
    engine = r"SP",                                               # Single precision.
    flags = ['-t'],                                               # Lets just test for now.
    async_runs = 2,                                               # Run two models at once.
    stdout = r'..\trash\stdout'                                   # Location to save the output.
)

Create the runs

In [3]:
S1 = ["Exg", "D01"]
S2 = ["5m"]
E1 = ["Q100", "QPMF"]
E2 = ["2hr", "4hr"]

runs = [tfr.Run(s1=s1, s2=s2, e1=e1, e2=e2) for s1 in S1 for s2 in S2 for e1 in E1 for e2 in E2]

Create the runner and stage the runs.

In [4]:
runner = tfr.Runner(parameters)
runner.stage(runs)
runner.get_runs()

[{'s1': 'Exg', 's2': '5m', 'e1': 'Q100', 'e2': '2hr'},
 {'s1': 'Exg', 's2': '5m', 'e1': 'Q100', 'e2': '4hr'},
 {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'},
 {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'Q100', 'e2': '2hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'Q100', 'e2': '4hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}]

Remove the {'Exg', '4hr'} and {'D01', '2hrs'} runs, there are not defined in the demostation.

In [5]:
runner.remove_runs(runner.get_runs('Exg', '2hr', any=False))
runner.remove_runs(runner.get_runs('D01', '4hr', any=False))
runner.get_runs()

[{'s1': 'Exg', 's2': '5m', 'e1': 'Q100', 'e2': '4hr'},
 {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'Q100', 'e2': '2hr'},
 {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'}]

Test the model, block on this cell so we dont move into the real run.

In [6]:
parameters.flags = ['-t', '-b']
runner.run()

executing: {'s1': 'Exg', 's2': '5m', 'e1': 'Q100', 'e2': '4hr'}_None
executing: {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}_None
executing: {'s1': 'D01', 's2': '5m', 'e1': 'Q100', 'e2': '2hr'}_None
executing: {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'}_None
---- ALL RUNS COMPLETE ----


Test is good, run the model.

In [7]:
parameters.flags = ['-b']
runner.run()

executing: {'s1': 'Exg', 's2': '5m', 'e1': 'Q100', 'e2': '4hr'}_None
executing: {'s1': 'Exg', 's2': '5m', 'e1': 'QPMF', 'e2': '4hr'}_None
executing: {'s1': 'D01', 's2': '5m', 'e1': 'Q100', 'e2': '2hr'}_None
executing: {'s1': 'D01', 's2': '5m', 'e1': 'QPMF', 'e2': '2hr'}_None
---- ALL RUNS COMPLETE ----


If we want to stop the runs, call the below method. 

In [ ]:
raise Exception("Go No Further!")

In [9]:
runner.stop()

That it's. Running models from python is easy, fast and unambiguous! Using Jupyter Notebooks it is easy to document your models as you develop and run them. This combined with Tuflow's own pytuflow reporting library and powerful GIS libraries like GDAL, fully automated workflows can be established. 